In [1]:
import deepcut
import numpy as np
from tensorflow.keras.preprocessing import sequence
from tensorflow import keras
from pythainlp import word_vector

In [2]:
with open('corpus.txt') as f:
    xtrain = []
    ytrain = []
    for line in f.readlines():
        temp = line.split()
        if len(temp) == 2:
            xtrain.append(temp[0])
            ytrain.append(temp[-1])
        elif len(temp) > 2:
            join = " ".join(temp[0:-1])
            xtrain.append(join)
            ytrain.append(temp[-1])
#         print(line)
print(xtrain)
print(len(xtrain))

['ส่วนบริเวณยอดดอยและยอดภูมีอากาศหนาว', 'ส่วนบริเวณยอดดอยและยอดภูมีอากาสหนว', 'ได้อ่อนกำลังลงเป็นหย่อมความกดอากาศต่ำกำลังแรงบริเวณใกล้ชายฝั่งประเทศเวียดนามตอนกลางแล้ว', 'ได้อ่อนกำลังลงเป็ยหย่อมความกดอากาสต่ำกำลังแรงบริเวณใกล้ชายฝั่งประเทศเวียตนามตอนกลงแล้ว', 'บริเวณภาคตะวันออกเฉียงเหนือตอนล่าง และภาคตะวันออก มีฝนเล็กน้อยถึงปานกลางกับมีลมแรง', 'บริฌวณถาคตะวันออกเฉียงเหนีอตอนล่าง และถาคตะวันออก มีฝนเล็กน้อยถึงปานกลษงกับมีลมแรง', 'บริเวณความกดอากาศสูงกำลังปานกลางหรือมวลอากาศเย็นจากประเทศจีนแผ่ปกคลุมประเทศไทยตอนบน', 'บริเวณคาวมกดอากาสสูงกำลังปานกลางหรือมวลอากาศเย็ฯจากประเทสจืนแผ่ปกคลุมประเทศไทยตอนบน', 'ได้เคลื่อนขึ้นฝั่งบริเวณดังกล่าวและคาดว่าจะอ่อนกำลังลงอีกเป็นหย่อมความกดอากาศต่ำในระยะต่อไป', 'ได้เคลิ่อนขึ้นฝั่งบริเวณดังกล่วและคาดว่าจะอ่อนกำลังลงอีกเป็ฯหย่อมความกดอากาศต่ำในระยะต่อไป', 'ทำให้บริเวณดังกล่าวมีอากาศเย็น โดยมีฝนบางแห่ง', 'ทำให้บริเวญดังกล่วมีอากาศเย็ฯโดยมีฝนบางแห่ง', 'ส่วนบริเวณยอดดอยและยอดภูมีอากาศหนาว สำหรับมรสุมตะวันออกเฉียงเหนือพัดปกคลุมอ่าวไทยและภาคใต้', 'ส่วนบริเวณยอดดอ

In [3]:
print(ytrain)
print(len(ytrain))

['ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด', 'ถูก', 'ผิด']
138


In [4]:
# x_train = ['วันนี้วันจันทร์','วันนี้วันศุกร์','วันนี้วันเสาร์','วันนีวันจันทนะ','วันนีวันศุกนะ','วันนีวันศุร์นะ','วันนี้วันศุกร์นะ','วันนี้วันจันทร์นะ',]
# y_train = ['ถูก','ถูก','ถูก','ผิด','ผิด','ผิด','ถูก','ถูก',]
# x_test = ['วันนี้วันพุธ','วันนี้วันอังคาร','วันนี้วันอังคาน','วันนี้วัลพุธ']

In [5]:
# corpus = ['วันนี้วันจันทร์','วันนี้วันศุกร์','วันนี้วันเสาร์','วันนีวันจันทนะ','วันนีวันศุกนะ',
#           'วันนีวันศุร์นะ','วันนี้วันศุกร์นะ','วันนี้วันจันทร์นะ','วันนี้วันพุธ','วันนี้วันอังคาร','วันนี้วันอังคาน','วันนี้วัลพุธ']

In [6]:
test = deepcut.tokenize('วันนี้วันศุร์นะ')
print(test)

['วัน', 'นี้', 'วัน', 'ศุร์นะ']


In [7]:
def word_tokenize(corpus):
    x = 0
    words = []
    for i in corpus:
        list_word = deepcut.tokenize(i,custom_dict='custom_dict.txt') #ตัดคำ
        words.append(list_word)
        x += 1
    return words
# corpus=word_tokenize(corpus)
# x_test = word_tokenize(x_test)
words = word_tokenize(xtrain)
# print(words)
# print(len(words))

In [8]:
def add_1list(word_list):
    list_ = []
    vocab = []
    for l in word_list:
        list_.extend(l)
#     print(list_)
    word_v = set(list_)
    for list_d in word_v:
        vocab.append(list_d)
    vocab = [' '] + vocab
    return vocab

vocab = add_1list(words)
print(vocab)
print(len(vocab))

[' ', 'เคลิ่อฯ', 'เมขเป็ฯ', 'พัด', 'เป็น', 'ตะวัน', 'นัก', 'ฯที่', 'ภากเหนือ', 'ดีเปรสชัน', 'ตอยบน', 'มาฏ', '70', 'ละ', 'รัมย', 'ดัง', 'ประเทสเวียดนาม', 'ผ่าน', 'สำหรัย', 'ให้', 'เทือกเขา', 'คะนอง', 'หภูมืลด', 'โดย', 'ตม', 'เขา', 'ภาก', 'ไป', '(', 'จังหวัดนครราชสีมา', 'สภาพ', 'ใต้', 'ระมัดระวัง', 'เขต', 'เซวเซียส', 'ชม.', 'อุณหภูม', 'ตลอก', 'หนว', 'สู่', 'สุรินทร์', 'กม.', 'ทะเลอันดามัน', 'ได้', 'ต่อ', 'กว่า', 'ด้วย', 'พื้นทื่', 'คาวม', 'เมตร', 'อุบลราธานี', 'อุณหภูมิ', 'เคลื่อน', 'ปานกลาง', 'ปกคลุม', 'ตอนบฯ', 'ชาย', 'ตะลอด', 'สะภาพ', 'ชาว', 'เข้า', 'สุดท้ย', 'ระลอก', 'ประกอบ', 'จืน', 'สุขภาพ', 'เมรต', 'กัมภูชา', '31', '15', 'เดิน', 'เพฉาะ', 'ปัตตานี', 'เมฆ', 'ร้อฯ', 'รายงาน', 'บุรีรัมย์', 'เร็ว', 'ลด', 'คาด', 'อุบลราชธานี', 'โชน', 'ขอ', 'จน', 'ก.ม.', 'เวียดนาม', 'แต่', 'อษกาศ', 'ประเทศ', '20', 'อันดามัน', 'อิทธิพล', 'อากาศ', 'ตัว', 'เกีอบ', 'ยอด', '1', 'ต่ำ', 'ขึ้รฝั่ว', 'ปาน', 'ฟน', 'อากาดเย็น', 'ศรีสะเกษ', 'หลย', 'กนาว', 'สัปดา', 'แรก', 'ชัน', 'พาดผ่าน', 'ความ', 'เหนีอ', 'กว้าง', 'ต

In [9]:
# words1 = []
# x = 0
# y = 0
# for text in range(len(words)):
#     for word in range(len(words[y])):
#         words1.append(words[y][x])
#         x += 1
#     y += 1
#     x = 0
# print(words1)
# words1 = set(words1)
# print(words1)
# # vocab = []
# # for list_d in words1:
# #     vocab.append(list_d)
# # vocab = [' '] + vocab
# # print(vocab)
# # print(len(vocab))

In [10]:
w2v = word_vector.get_model()

In [11]:
vecter = []
vocab_ = []
index = 0
for word in w2v.index2word:
    vocab_.append(word)
    vecter.append(w2v[word])
#     thai2dict[word] = w2v[word]
#     print(word)

In [12]:
vocab_ = [''] + vocab_
vecter = np.vstack((np.random.rand(*vecter[0].shape), vecter))

In [13]:
print(vocab_[0])
print(len(vocab_))


51359


In [14]:
# print(vecter[0])

In [15]:
def sent2idx(x):
    xidx = []
    for w in x:
#         print(w)
        if w in vocab_:
            xidx.append(vocab_.index(w))
        else:
            xidx.append(0)
    return np.array(xidx)

In [16]:
sent2idx(words[1])

array([   98,   213,   749,  2922,     2,   749, 26023,     0,     0])

In [17]:
def word_seq(words):
    Xtrain_idx = []
    maxlen = 0
    for x in words:
        Xtrain_idx.append(sent2idx(x))
        if len(Xtrain_idx[-1]) > maxlen:
            maxlen = len(Xtrain_idx[-1])
#         print(len(Xtrain_idx[-1]))
    print(maxlen)
    return Xtrain_idx

In [18]:
# Xtest = word_seq(x_test)
# print(Xtest)
Xtrain = np.array(word_seq(words))
print(Xtrain)

34
[array([   98,   213,   749,  2922,     2,   749, 26023,  1162,  6524])
 array([   98,   213,   749,  2922,     2,   749, 26023,     0,     0])
 array([    6,  1328,   332,   110,     3, 16599,   100,  1858,  1162,
        1314,   332,  1025,   213,   677,   390,  1080,    71,   888,
         318,   348,    62])
 array([    6,  1328,   332,   110, 38101, 16599,   100,  1858,     0,
        1314,   332,  1025,   213,   677,   390,  1080,    71, 31507,
         318,     0,    62])
 array([ 213,  276, 3606,   82, 8184,  454,  318, 1850,    0,    2,  276,
       3606,   82,    0,    5, 3222, 1450,   72, 4938,   14,    5,  811,
       1025])
 array([   0,    0, 3606,   82, 8184,    0,  318, 1850,    0,    2,    0,
         82,    0,    5, 3222, 1450,   72,    0,   14,    5,  811, 1025])
 array([ 213,  100, 1858, 1162,  258,  332, 4938,   31, 2067, 1162, 1930,
         18,   71,  320, 4330, 3240,   71,   86,  318,  135])
 array([ 213,    0, 1858,    0,  258,  332, 4938,   31, 2067, 1162, 

In [19]:
max_len = 40
Xtrain = sequence.pad_sequences(np.array(Xtrain),maxlen=max_len,padding='pre')
print(Xtrain)

[[    0     0     0 ... 26023  1162  6524]
 [    0     0     0 ... 26023     0     0]
 [    0     0     0 ...   318   348    62]
 ...
 [    0     0     0 ...     0   164     0]
 [    0     0     0 ...  1198   927  1043]
 [    0     0     0 ...  1198   927     0]]


In [20]:
Xtrain.shape

(138, 40)

In [21]:
intput_lenght = len(Xtrain[0])
print(intput_lenght)
vocab_size = len(vocab)+1

40


In [22]:
# Xtest = sequence.pad_sequences(np.array(Xtest),maxlen=intput_lenght,padding='pre')
# print(Xtest)

In [23]:
y_train = []
for i in ytrain:
    if i == 'ถูก':
        y_train.append(1)
    else:
        y_train.append(0)
print(y_train)

[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]


In [24]:
model = keras.Sequential()
model.add(keras.layers.Embedding(vecter.shape[0], vecter.shape[1], name='embed'))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True)))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(12)))
model.add(keras.layers.Dense(64, activation=keras.activations.relu))
model.add(keras.layers.Dense(2))

In [25]:
model.compile(loss=keras.losses.sparse_categorical_crossentropy, optimizer=keras.optimizers.Adam(learning_rate=1e-4),metrics=['accuracy'])

In [26]:
model.get_layer('embed').set_weights([vecter])
model.get_layer('embed').trainable = False

In [27]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, None, 300)         15407700  
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         186880    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 24)                13536     
_________________________________________________________________
dense_14 (Dense)             (None, 64)                1600      
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 130       
Total params: 15,609,846
Trainable params: 202,146
Non-trainable params: 15,407,700
_________________________________________________________________
None


In [34]:
model.fit(Xtrain, np.array(y_train), epochs=20,validation_split=0.1)

Epoch 1/20
4/4 [==============================] - 0s 89ms/step - loss: 0.6931 - accuracy: 0.4919 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/20
4/4 [==============================] - 0s 79ms/step - loss: 0.6931 - accuracy: 0.4919 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/20
4/4 [==============================] - 0s 91ms/step - loss: 0.6931 - accuracy: 0.4919 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/20
4/4 [==============================] - 0s 89ms/step - loss: 0.6931 - accuracy: 0.4919 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 5/20
4/4 [==============================] - 0s 89ms/step - loss: 0.6931 - accuracy: 0.4919 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 6/20
4/4 [==============================] - 0s 85ms/step - loss: 0.6931 - accuracy: 0.4919 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 7/20
4/4 [==============================] - 0s 87ms/step - loss: 0.6931 - accuracy: 0.4919 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 8/20
4/4 [===========

In [35]:
score = model.evaluate(Xtrain,np.array(y_train),verbose=1)
print(score)

5/5 [==============================] - 0s 4ms/step - loss: 0.6931 - accuracy: 0.4928
[0.6931471824645996, 0.49275362491607666]


In [36]:
model.predict(Xtrain).argmax(axis=1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0], dtype=int64)

In [46]:
# model.predict(Xtest).argmax(axis=1)